In [ ]:
%pip install chromadb

In [ ]:
%pip install chroma-hnswlib langchain-chroma

In [4]:
from langchain_community.document_loaders import PyPDFLoader
from langchain.vectorstores import Chroma
from langchain_ollama import OllamaEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter


In [2]:


embeddings = OllamaEmbeddings(
    #model="llama3.2:latest",
    model="llama3.2:latest",
    base_url="http://localhost:11434"  # Ensure Ollama is running locally
)




In [ ]:

# Step 2: Load PDF and Split into Chunks
loader = PyPDFLoader("../../00-example_data/layout-parser-paper.pdf")
pdf_docs = loader.load()

print(f"Loaded {len(pdf_docs)} documents from the file.")

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=20,
    length_function=len,
    is_separator_regex=False,
)
documents = text_splitter.split_documents(pdf_docs)


In [ ]:
# Create Chroma vector store

vector_store = Chroma.from_documents(documents, embeddings, persist_directory="./chroma_db")

# Persist data to disk
vector_store.persist()


In [7]:

# Reload persisted data
new_vector_store = Chroma(persist_directory="./chroma_db", embedding_function=embeddings)

In [ ]:
# Step 5: Perform a Query
query = "logic in the document?"
results = new_vector_store.similarity_search(query, k=3)

print("\nMost Similar Documents:")
for idx, result in enumerate(results, start=1):
    print(f"{idx}. {result.page_content}")